# Implication of competition on ecological response to warming

# Bacteria community model: competition, warming and size

Motivation: To examine how microbial communities and ecosystem function will respond to warming and disturbance (e.g. chemical pollution such as pesticides or antibiotics). This model investigates the impact of response of bacteria to warming based on different functional groups competing for the same resources. This will add to the small body of literature which emphasises that: 'food web response to temperature cannot be derived from the single species temperature response' [(Nomdedeuet al. 2013)](https://link.springer.com/content/pdf/10.1007%2Fs00227-012-1966-x.pdf). In most food web/low trophic/biogeochemical models changes to species/functional group interactions with warming are not considered.

This model builds on the work by [Marsland et al. (2019)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1006793), which uses the seminal MacArthur (1970) species competition model. It includes a consumer pool (bacteria) and a resource pool (organic compounds). The bacteria can feed on specific resources only, and the resources consumed by the bacteria can be converted into other resources. Multiple bacteria with different functional traits (e.g. size, response to warmings (Thermal Performance Curves, TPCs), nutrient preference) compete for the same resourse. This notebook sets out the model and determines which traits are selected for under different environment and community scenarios.

For detailed model description and explanation of the equations see this [notebook](https://nbviewer.jupyter.org/github/CleggTom/Bacteria_EcoFunc/blob/master/notebooks/Metabolic_model.ipynb) by Tom Clegg who built the initial model. 


## Model set up

### Consumers

For an $i$th consumers with $j$ resources the core consumer equation is:

\begin{equation}
dC_i/dt = C_i\Bigl(\sum_{j=1}^{M}u_{ij}S_j(1-Rg_j-\sum_{k=1}^{M}l_{jk}) - Rm_i\Bigl) \tag{1}
\end{equation}

where $C_i$ is the biomass of consumer $i$, $u_{ij}$ is the preference of consumer $i$ to uptake resource $j$, $S_j$ is the mass or concentration of resources $j$, $Rg_j$ is the loss of consuming resource $j$ to respiration, $l_{jk}$ is the proportion of $j$ taken up by the bacteria that is converted to and released as resource $k$, and $Rm_i$ is the maintenance cost loss term. 

### Resources

For a $j$th resource the core equation is:

\begin{equation}
dS_j/dt = \rho_j - \sum_{i=1}^{N}\Bigl(C_iu_{ij}S_j-\sum_{k=1}^{M}C_iu_{ij}S_jl_{kj}\Bigl) \tag{2}
\end{equation}

where $\rho$ is the external input of resource $j$ and $l_{kj}$ is the gain and conversion of resource $j$ from $k$.

### Constraints

\begin{array}{|c|c|c|}
\hline Parameter/Variable & Dimension & Units & Constraint & Temp/size-dependant & Initial-value \\\hline
  External-nutrient-supply, \rho_j & vector (,j) & mass/time & > 0 & no & 1 \\\hline
  Nutrient-uptake, u_{ij} & matrix (i,j) & /masstime & > 0& yes & 1 \\\hline
  Growth-respiration, Rg_j & vector (,j) & NA & < 1 & no & 0.2 \\\hline
  Maintenance-respiration, Rm_i & vector (i,) & /time & >0, < 1& yes & 0.5 \\\hline
  Excretion-to-new-nutrient, l_{jk} & matrix (j,k) & NA & < 1& no & 0.4 \\\hline
  Nutrients& vector (j,) & mass& > 0 & NA & 1 \\\hline
  Bacteria & vector (i,) & mass& > 0 & NA & 0.1 \\\hline
\end{array}

The above table sets our the contraints on the parameters and variables (nutrients and bacteria). There is another condition too that growth respiration and excretion must not sum to 1 or more ($Rg_j$ + $l_{jk}$ < 1) as otherwise no nutrient is gained by the bacteria.



### Comparison to the Marsland model

The equations below set out the Marsland 2019 model.

\begin{equation}
dC_i/dt = g_iC_i\Bigl(\sum_{j=1}^{M}w_j\sigma u_{ij}S_j(1-\sum_{k=1}^{M}l_{jk}) - Rm_i\Bigl) \tag{3}
\end{equation}

\begin{equation}
dS_j/dt = \rho_j + \sum_{i=1}^{N}C_i\Bigl(\sum_{k=1}^{M}\sigma u_{ij}S_j l_{kj} - \sigma u_{ij}S_j\Bigl) \tag{4}
\end{equation}

where $g_i$ is the proportional constant between per capita growth rate and energy flux, $w_j$ is the energy density of resource $j$ (units are energy/mass) and $\sigma$ is the functional response (units mass/time).

#### Differences between ours and Marslands model

- We do not parameterise energy density of resource ($w_j$) and therefore nor the constant between growth and energy flux ($g_i$) 
- We assume a Type I functional response ($\sigma$)
- For Marsland $\rho_j$ = 0 except for $\rho_0$, so only the first resource has an external input which then diversifies. In our model all resources have an input.
- We include the addition of respiration loss associated with growth ($R_g$) as a loss term in the Bacterial mass (dC/dt) equation.

## 2-species 2-resources model 

### Bacteria

\begin{equation}
dC_1/dt = C_1\Bigl(\Bigl(u_{11}S_1(1-Rg_1-(l_{12}+l_{21})) + u_{12}S_2(1-Rg_2-(l_{21}+l_{12}))\Bigl) - Rm_1\Bigl) \tag{5}
\end{equation}

\begin{equation}
dC_2/dt = C_2\Bigl(\Bigl(u_{21}S_1(1-Rg_1-(l_{12}+l_{21})) + u_{22}S_2(1-Rg_2-(l_{21}+l_{12}))\Bigl) - Rm_2\Bigl) \tag{6}
\end{equation}

### Resources
\begin{equation}
dS_1/dt = \rho_1 - \Bigl((C_1u_{11}S_1-   (C_1u_{11}S_1l_{21} + C_1u_{11}S_1l_{12})) + (C_2u_{21}S_1-   (C_2u_{21}S_1l_{21} + C_2u_{21}S_1l_{12}))\Bigl) \tag{7}
\end{equation}

\begin{equation}
dS_2/dt = \rho_2 - \Bigl((C_1u_{12}S_2-   (C_1u_{12}S_2l_{12} + C_1u_{12}S_2l_{21})) + (C_2u_{22}S_2-   (C_2u_{22}S_2l_{12} + C_2u_{22}S_2l_{21}))\Bigl) \tag{8}
\end{equation}



![alt text](Figure_2_species_equil.png)

This figure shows the a model run with 2 species (green) and 2 resources (blue). Each species prefers one resouce such that $u_{ij}$ = ([1,0], [0,1]). As there is no competition both species have the same equilibrium mass as do both resources. The starting parameters are given in Table 1.

### Co-existence at equilibria

The solutions for bacteria at equilibrium are:

\begin{equation}
C = 0
\end{equation}

\begin{equation}
C = \frac{\rho(1 - Rg - l)}{Rm(1-l)} = \frac{1(1 - 0.2 - 0.4)}{0.5(1-0.4)} = 1.33 \tag{9} 
\end{equation}

and for resources are:

\begin{equation}
S = 0
\end{equation}

\begin{equation}
S = \frac{Rm}{u(1-Rg-l)} = \frac{0.5}{1(1-0.2-0.4)} = 1.25 \tag{10}
\end{equation}


## R*

To determine which species will win we can calculate the $R*$, the amount of resource needed for a species to remain at equilibrium. To do this we insert the equilbrium equation for bacteria ($C$, Eq 9) into the equation for bacteria ($C$, Eq 1) and solve for S:

\begin{equation}
0 = \frac{uS\rho(1 - Rg - l)(1-Rg-l)}{Rm(1-l)} - Rm 
\end{equation}

\begin{equation}
uS\rho(1 - Rg - l)^2 = Rm^2(1-l) 
\end{equation}

\begin{equation}
R*  = \frac{Rm^2(1-l)}{u\rho(1 - Rg - l)^2} \tag{11}
\end{equation}


Both nutrient uptake ($u$) and maintenance respiration ($Rm$) can be used to define functional groups based on their temperature dependancies.


- When T = T_pk the difference in R* between different individuals is largest.
- Groups with lower $Ea$ have competitive advantage
- The assembly model (below) selects for species with low $Ea$


![alt text](R*_20C.png)

<!--- Plotting bacteria mass against R* for the structured and random preference matrix gives:--->


<!--- ![alt text](B_vs_R_struct.png)--->

<!--- ![alt text](B_vs_R_rand1.png)--->

<!--- Where purple = low temperature and yellow = high temperature. Green is T_pk (20 C) which gives highest R* and B.--->


<!--- Also, when randomly sharing nutrients do not have same mass of nutrient that their competition (see nutrient (N_num) #8 in table).--->
<!--- ![alt text](preview_data.png)--->


## Assembly model

If individuals die new species with randomly selected functional traits can replace them. In this model death and therefore immigration/assembly only occurs if there is competition between bacteria for nutrients. If there is no competition all bacteria can survive. Below is at 4C.

![alt text](Rplot01.png)

In the above plot only activation energy is changed, left Ea is same, right Ea is random (0.5-0.9)
When no competition and all B have same Ea (light green left plot) then total B concentration is higher, when there is competition (2 B per N) then the concentration is halfed. As all function same no change in B with time and all B survive.
When function (Ea) changes then get diff amounts of N and B. When comp added (2 B per N) then half die off. In assembly those that have died will be replaced. Now nutrients change too.

Need both competition AND different functional traits for to cause some species to die off. If you have just one of those all bacteria can survive.

### Modelling functional traits

#### Temperature and size dependance

In this model only $u_{ij}$ and $Rm_i$ can be temperature- and size-dependant as they are the only terms that explicitly depend on an individual consumer $i$. Temperature- and size-dependancy are formulated following the Metabolic Theory of Ecology and the Sharpe-Schoolfield equation. Such that:

\begin{equation}
u_{ij} | Rm_i = BM^{\frac{3}{4}} * \frac{e^{\frac{-E}{k}\cdot\Bigl(\frac{1}{T} - \frac{1}{T_{ref}}\Bigl)}}{1 + \frac{E}{E_D - E}\cdot e^{\frac{-E_D}{k}\cdot\Bigl(\frac{1}{T_{pk}} - \frac{1}{T}\Bigl)}} \tag{12}
\end{equation}


where $BM^{0.75}$ is the mass ($M$) scaled to $3/4$ multiplied by a constant that is higher for uptake ($u_{ij}$) than maintenance respiration ($Rm_i$). The second part of the equation is the Sharpe-Schoofield equation where $E$ is the activation energy in eV, $E_D$ is the deactivation energy after the temperature maximum metabolism is reached ($T_{pk}$), $T$ is the temperature and $T_{ref}$ is the reference temperature. All temperatures are expressed in Kelvin (K) and $k$ is the Boltzmann constant of $8.62 x 10^{-5} eV K^{-1}$.

#### Expressing size dependancy

It is uncertain whether size would increase fitness/population size in bacteria. This is because a larger size could increase genome increasing growth rate, but could also increase maintenance respiration ($Rm_i$) which would negatively impact on growth rates and population size. Instead of forcing size to have a positive or negative impact on population size both these relationships can be incorporated into the model. As body size increases, $u_{ij}$ will rise, increasing uptake and have a positive impact on population growth. $Rm_i$ will also increase with body size, but this is a loss term from the bacteria consumer so decreases overall population growth. $B$ for $u_{ij}$ (growth, $Bg$) will be higher than for $Rm_i$ (respiration, $Brm$).

## Context and possible aim

### Aim
Publish a model that can predict which functional groups (> 2, i.e. complex communities) will win/emmigrate in novel environmental conditions caused by warming. Show how R* varies with temperature sensitivity (Ea and Tpk)

In this laboratory study [(Jiang & Moran 2004)](https://besjournals.onlinelibrary.wiley.com/doi/full/10.1111/j.0021-8790.2004.00830.x) two cilitates are grown independantly at different temperatures they grow without an issue (data not shown). When they grow together at different temperatures one (the specialist, closed points) outcompetes the other (the generalist, open points) at intermediate temperatures. 

<img src="Cilitates.png" alt="Kitten" title="" width="400" height="300" />

Also see 'Our model system illustrates that horizontal (competition) and vertical (enemy–victim) interactions and dispersal markedly affect the distributions and abundances of species. Because these factors are omnipresent in nature, predicting changes in distribution and abundance under global warming by extrapolation of the climate envelope may lead to serious errors' from [(Davis et al 1998)](https://www.nature.com/articles/35842). 

And 'The species interaction mechanism both helps to explain why many treelines have not advanced in response to climatic warming and highlights that predictions of treeline shifts based solely on climate may be misleading, because interspecific interactions can temper effects of climatic change'. [(Liang et al 2016)](https://www.pnas.org/content/113/16/4380.short)

---------------------------------------------------------------------------------------------------------------------

|

|

|

|

|

|

|


---------------------------------------------------------------------------------------------------------------------

## Results

### Competition

Bacteria compete for the same resources as others that likely have different functional traits to them. Not impacted by $l_{jk}$ term - is it too small or because in vector notation? Whomever is best adapted to that environment, either by being larger or by having a better temperature sensitivity will outcompete the other. The only way competition that finalises in some species dying in the model is if there are different functional traits for different individuals. 

#### Functional trait 1: temperature sensitivity of metabolic rate

i.e. Changing the activation energy. 

##### Initial results
Below are the end results (10th run) of mean (n = 50 per temperature) bacterial mass at different temperatures. The colours are the activation energies ranging from 0.5 - 0.9. De-activation is set to 1.5 and Tref = 0 C. The left plots have the same number of species and nutrients so there is no competition. The right plots have half the number of nutrients introducing competition and im/emigration. The top plots have a Tpk of 12 and the bottom a Tpk of 20. 

The effects of temperature are the same for all species. Some individuals may become more or less competitive in reality.

![alt text](TPC_comp_and_no-comp_same_scale.png)

Competition dampens the impact of warming.

##### Initial results with error bars (standard deviation)
This shows without error bars, introducing error bars to all plots highlights the 'randomness' of the associations of which there are 15 possible combintations of bacteria $Ea$ per nutrient. There are 50 bacteria in this model run, I would need to add more bacteria (without increasing possible combinations) to reduce the error bars in the competition model. There is no error in the no-competition model.

![alt text](TPC_comp_and_no-comp_error_same_scale.png)

The plot below is the mean bacterial mass at end of 10 runs for all bacteria at that temperature, compared to above plot where the bacteria are split by $Ea$.

![alt text](TPC_all_B_same_scale.png)



##### Selection for low temperature sensitivies
It is also helpful to look at the distribution of temperature senstivities of the bacteria in the model by the end of the model runs under competition and without competition. The below plot is the results from 4C.

![alt text](Ea_count_4C.png)

After 10 runs if there is no competition then no bacteria dies, so there is no sorting of individuals with a certain $Ea$, i.e. there is no em- and immigration. In the competition models after 10 runs there are always more bacteria that have a lower $Ea$, so higher $Ea$ are dying and being removed. 

##### Drivers of bacterial mass in competition model
To explore what is causing these trends I investigate the relationship between an individual bacteria's $Ea$ and it's direct competitors $Ea$ and plotted that against the first individuals bacterial mass. This is data from the 4 C model run but the same patterns exists at all temperatures: bacteria with a lower $Ea$ than their competitor will have a higher bacterial biomass. Absolute $Ea$, $S$ and the difference in $Ea$ all contribute to driving bacterial mass. IN the plot below the colours refer to the absolute $Ea$ value and the size of points nutrient mass ($S$).

![alt text](Competitors_Ea.png)


#### Functional trait 2: Size of bacteria and temperature

Temperature sensitivity is proportional to size...is this correct?

![alt text](size_and_temp.png)




### Sensitivity analysis

To determine which parameters are driving model results I varied the temperature sensitivity ($Ea$), size impact on growth ($Bg$) and size impact on metabolism respiration ($Brm$). The graph shows the change in bacteria mass relative to the 'Control' run at 4 different temperatures.

\begin{array}{|c|c|c|c|}
\hline Experiment & Ea & Bg & Brm \\\hline
  Control& 0.7 & 0.5 & 0.25 \\\hline
  Low-Ea& 0.5 & 0.5 & 0.25 \\\hline
  High-Ea& 0.9 & 0.5 & 0.25 \\\hline
  Low-Bg& 0.7 & 0.3 & 0.25 \\\hline
  High-Bg& 0.7 & 0.7 & 0.25 \\\hline
  Low-Brm & 0.7 & 0.5 & 0.05 \\\hline
  High-Brm & 0.7& 0.5 & 0.25 \\\hline
\end{array}

![alt text](Sensitivity_analysis.png)

Both $Ea$ and $Bg$ had a postive impact on mass, i.e. higher values than the control increased mass. $Brm$, the size component for maintenance respiration (larger organisms have high respiratory cost) decreased mass compared to the control. The size effect on respiration is the most sensitive parameter, causing the largest offest from the control biomass at -0.07 at 10 C. In this analysis, in each experiment all bacteria are parameterised the same, so there is no assembly as all can survive together. It may be worth re-doing the sensitivity analysis allowing deaths to occur.

The fact respiration has the largest influence could be why lower Ea's and sizes are selected for in the competition runs, as high Eas and large sizes lose out to lower Eas and sizes.

## Parameterising using Tom Smith's data

Ea = 0.6 - 1.0  
Ea_D = 3.5  
B0_g = e$^{(2.5 + (-5 * Ea))}$  
B0_rm = 0.5 * B0_g  
T_pk = 20

## Changing resource preference

All run to steady state.

#### Resource preference structured. 1 resource per bactera.

Top left = Only Ea varies  
Top right = Ea and B0 (g and rm) vary  
Bottom left = Ea and B0_g vary  
Bottom right = Ea and B0_rm vary

![alt text](Smith_diag_pref.png)


#### Resource preference random. 1 resource per bactera.

Top left = Only Ea varies  
Top right = Ea and B0 (g and rm) vary  
Bottom left = Ea and B0_g vary  
Bottom right = Ea and B0_rm vary

![alt text](Smith_rand_pref_1.png)

#### Resource preference random. 1 resource per bactera.

Top left = Only Ea varies  
Top right = Ea and B0 (g and rm) vary  
Bottom left = Ea and B0_g vary  
Bottom right = Ea and B0_rm vary

![alt text](Smith_rand_pref_2.png)


But resources spiking either when use random preference matrix

#### Structured preference matrix
![alt text](Figure_ein_exist_2.png)

#### Random preference matrix
![alt text](Figure_ein_exist.png)
